**Authentication** 

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
def load_file_from_drive(id, name):
    downloaded = drive.CreateFile({'id':id})
    downloaded.GetContentFile(name,mimetype='text/csv')

**imports**

In [4]:
import time

import pandas as pd
import numpy as np

from operator import or_
import functools

**load sudoku examples data**

1 million Sudoku games from [kaggle](https://www.kaggle.com/bryanpark/sudoku/data#)

In [5]:
load_file_from_drive('1mVFwkSB4Y-VHGVIn0aTiBrx-Hk5YBo0Z', 'sudoku.csv')

In [6]:
sudokus = pd.read_csv('sudoku.csv')

**backtracking with mrv and degree heuristics**

In [7]:
def create_grid(string):
    grid = []
    for i in range(0,len(string),9):
        row = [int(i) for i in string[i:i+9]]
        grid.append(row)
    return grid

In [8]:
def get_square(grid,row,col):
    sq_row = (row//3)*3
    sq_col = (col//3)*3
    sq=[]
    for i in range(sq_row,sq_row+3):
        for j in range(sq_col,sq_col+3):
            sq.append(grid[i][j])
    return sq

In [9]:
def is_consistent(grid,row,col,num):
    width = len(grid)
    for i in range(width):
        if grid[i][col] == num or grid[row][i] == num: return False

    if num in get_square(grid,row,col) : return False

    return True

In [10]:
def is_complete(grid):
    for row in grid:
        for var in row:
            if var == 0: return False
    return True

In [28]:
def get_domain(grid,row,col):
    '''
    returns domain(consistent values) of a position
    '''
    values = list(range(1,len(grid)+1))
    domain = []
    for n in values:
        if is_consistent(grid,row,col,n):
            domain.append(n)
    return domain

In [12]:
def mrv_domains(grid):
    '''
    returns coordiantes of cells 
    which have minimum-remaining-values (smallest domains)
    '''
    domains_dict = {}
    for i in range(len(grid)):
        for j in range(len(grid)):
            if grid[i][j] == 0:
                domains_dict[i,j] = get_domain(grid,i,j)

    min_remain_val = None

    for domain in domains_dict.values():
        if min_remain_val is None:
            min_remain_val = len(domain)
            continue
        min_remain_val = min(min_remain_val, len(domain))

    if min_remain_val == 0: return None #sudoku cannot be solved

    min_domains = {k: domains_dict[k]
                   for k in domains_dict.keys()
                   if len(domains_dict[k]) == min_remain_val}
    return min_domains

In [13]:
def get_degree(grid, row, col):
    '''
    returns number of cells in a given cell's corresponding row,column and block
    which are not assigned yet.
    '''
    degree = 0
    for i in range(0,len(grid)):
        if grid[i][col] == 0: degree+=1
        if grid[row][i] == 0: degree+=1
    for var in get_square(grid, row, col):
        if var == 0: degree+=1
    return degree

In [14]:
def get_max_degree(blocks):
    '''
    returns coordinates of the cell with maximum degree
    '''
    max_degree = None
    row = None
    col = None
    for i,j in blocks:
        degree = get_degree(grid,i,j)

        if max_degree is None:
            max_degree = degree
            row, col = i,j
            continue

        if degree > max_degree:
            max_degree = degree
            row, col = i,j
    return row, col

In [15]:
def select_unassigned_var(grid):
    '''
    select a cell for assigning based on mrv and degree heuristics.
    '''
    min_domains = mrv_domains(grid)

    if not min_domains:
        return None, None, []

    if len(min_domains) == 1:
        (row,col),dom = min_domains.popitem()
        return row, col, dom
    
    row, col = get_max_degree(min_domains.keys())

    dom = min_domains[row,col]
    return row, col, dom

In [16]:
def backtracking_search(grid):

    if is_complete(grid):
        return grid

    row, col, domain = select_unassigned_var(grid)

    for val in domain:
        
        if not is_consistent(grid, row, col, val): continue
        
        grid[row][col] = val
        result = backtracking_search(grid)
        if result:
            return result
        grid[row][col] = 0
    return False

**run**

In [17]:
sudoku_list = list(sudokus['quizzes'])[:100]
solutions = list(sudokus['solutions'])[:100]
bt_solutions = []

In [18]:
times = []

for sudoku in sudoku_list:
    grid = create_grid(sudoku)
    
    s_time = time.time()

    bt_solutions.append(backtracking_search(grid))

    times.append(time.time() - s_time) 

for i in range(len(bt_solutions)):
    sol_str = ''
    for row in bt_solutions[i]:
        for var in row:
            sol_str+=str(var)
    bt_solutions[i] = sol_str

In [19]:
# total time
sum(times)

8.440193176269531

In [20]:
from statistics import mean
# avg time for each sudoku
mean(times)

0.08440193176269531

In [21]:
bt_solutions == solutions

True